# Team Members

| Name | Contribution |
|------|--------------|
| Oindreela Bhowmick |  100% |
| Vaibhav Mehta | 100% |
| Thirumurugan R | 100% |

In [46]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
import sys
print(sys.version)

3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]


In [48]:
import pyspark
print(pyspark.__version__)

3.5.5


In [49]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType
from pyspark.sql.functions import *

In [50]:
#creating a sparksession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("LoadDriveData") \
    .getOrCreate()

In [51]:
#defining schema of the files
#since date column has a space in between, reading it as 2 columns date and timezone
schema = StructType([
    StructField("remotehost", StringType(), True),
    StructField("rfc931", StringType(), True),
    StructField("authuser", StringType(), True),
    StructField("date", StringType(), True),
    StructField("timezone", StringType(), True),
    StructField("request", StringType(), True),
    StructField("status", IntegerType(), True),
    StructField("bytes", IntegerType(), True)
])

In [52]:
file_path_jul = "/content/drive/My Drive/BDS_Assignment/Access_log_Jul1995.gz"
file_path_aug = "/content/drive/My Drive/BDS_Assignment/Aaccess_log_Aug1995.gz"

df_jul = spark.read.format("csv").option("delimiter", " ").schema(schema).load(file_path_jul)
df_aug = spark.read.format("csv").option("delimiter", " ").schema(schema).load(file_path_aug)

### 2. Data Wrangling

Issues -
1. Found malformed records in july logs, so dropped it to not affect our query results.
2. Since the delimiter of the files is space, the date column was split into 2 colums - date and timezone, due to having space between them.
Solution - we define a custom schema which reads the date column as 2 columns date and timezone, we drop the timezone column as provided in problem statement, and remove the extra [ present in the date column for further processing.
3. No valid data present in 2 columns, rfc931 and authuser.
Solution - dropping the columns to optimize query processing time.
4. Given data has duplicate records.
Solution - applying dropDuplicates to reduce query processing time.





In [53]:
print(f"July Count - {df_jul.count()}")
print(f"August Count - {df_aug.count()}")

July Count - 1891715
August Count - 1569898


In [54]:
log_data = df_jul.union(df_aug)

In [55]:
log_data.count()

3461613

In [56]:
log_data.printSchema()

root
 |-- remotehost: string (nullable = true)
 |-- rfc931: string (nullable = true)
 |-- authuser: string (nullable = true)
 |-- date: string (nullable = true)
 |-- timezone: string (nullable = true)
 |-- request: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- bytes: integer (nullable = true)



In [57]:
log_data.show(5,0)

+--------------------+------+--------+---------------------+--------+------------------------------------------------------------+------+-----+
|remotehost          |rfc931|authuser|date                 |timezone|request                                                     |status|bytes|
+--------------------+------+--------+---------------------+--------+------------------------------------------------------------+------+-----+
|199.72.81.55        |-     |-       |[01/Jul/1995:00:00:01|-0400]  |GET /history/apollo/ HTTP/1.0                               |200   |6245 |
|unicomp6.unicomp.net|-     |-       |[01/Jul/1995:00:00:06|-0400]  |GET /shuttle/countdown/ HTTP/1.0                            |200   |3985 |
|199.120.110.21      |-     |-       |[01/Jul/1995:00:00:09|-0400]  |GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0   |200   |4085 |
|burger.letters.com  |-     |-       |[01/Jul/1995:00:00:11|-0400]  |GET /shuttle/countdown/liftoff.html HTTP/1.0                |304   

To check if all the records have unavailable information for rfc931 and authuser

In [58]:
log_data.filter("trim(rfc931) != '-'").show(10,0)

+----------+------+--------+----+--------+-------+------+-----+
|remotehost|rfc931|authuser|date|timezone|request|status|bytes|
+----------+------+--------+----+--------+-------+------+-----+
+----------+------+--------+----+--------+-------+------+-----+



In [59]:
log_data.filter("trim(authuser) != '-'").show(10,0)

+----------+------+--------+----+--------+-------+------+-----+
|remotehost|rfc931|authuser|date|timezone|request|status|bytes|
+----------+------+--------+----+--------+-------+------+-----+
+----------+------+--------+----+--------+-------+------+-----+



we do not have authuser or rfc931 information available in given log data, so dropping these columns.

In [60]:
log_data = log_data.drop("rfc931","authuser")

Checking duplicate records in data

In [61]:
log_data.count() == log_data.dropDuplicates().count()

False

In [62]:
log_data = log_data.dropDuplicates()

In [63]:
log_data.count()

3460246

Parsing date column

In [64]:
# dropping the timezone column as given in question
log_data = log_data.drop("timezone")

In [65]:
# remove the [ from the date column
log_data = log_data.withColumn("date", regexp_replace("date", "\\[", ""))

In [66]:
log_data.select("date").show(10,0)

+--------------------+
|date                |
+--------------------+
|01/Jul/1995:00:02:57|
|01/Jul/1995:00:03:33|
|01/Jul/1995:00:07:30|
|01/Jul/1995:00:07:36|
|01/Jul/1995:00:09:18|
|01/Jul/1995:00:10:04|
|01/Jul/1995:00:11:32|
|01/Jul/1995:00:11:40|
|01/Jul/1995:00:17:52|
|01/Jul/1995:00:22:32|
+--------------------+
only showing top 10 rows



In [67]:
#convert the date column to datetype or timestamptype
log_data = log_data.withColumn("date2", to_timestamp("date", "dd/MMM/yyyy:HH:mm:ss"))

In [68]:
log_data.show(10,0)

+------------------------+--------------------+----------------------------------------------------------------+------+-----+-------------------+
|remotehost              |date                |request                                                         |status|bytes|date2              |
+------------------------+--------------------+----------------------------------------------------------------+------+-----+-------------------+
|waters-gw.starway.net.au|01/Jul/1995:00:02:57|GET /shuttle/missions/51-l/images/ HTTP/1.0                     |200   |1038 |1995-07-01 00:02:57|
|midcom.com              |01/Jul/1995:00:03:33|GET /history/apollo/apollo-13/apollo-13-patch-small.gif HTTP/1.0|200   |12859|1995-07-01 00:03:33|
|asp.erinet.com          |01/Jul/1995:00:07:30|GET /images/KSC-logosmall.gif HTTP/1.0                          |200   |1204 |1995-07-01 00:07:30|
|teleman.pr.mcs.net      |01/Jul/1995:00:07:36|GET /shuttle/missions/sts-71/images/images.html HTTP/1.0        |200   |7634 

In [69]:
#to check whether any valid date has not been converted properly
log_data.filter("date is not null and date2 is null").show(10,0)

+----------+----+-------+------+-----+-----+
|remotehost|date|request|status|bytes|date2|
+----------+----+-------+------+-----+-----+
+----------+----+-------+------+-----+-----+



checking if there are malformed/null records

In [70]:
log_data.filter(col("date").isNull()).count()

1

In [71]:
log_data.filter((col("date").isNull()) & (col("request").isNull()) & (col("status").isNull())).count()

1

since we found a record where most of the columns are null, we can drop this record.

In [72]:
log_data = log_data.filter(col("date").isNotNull())

In [73]:
#verifying whether only 1 record is dropped
log_data.count()

3460245

### 3. Analytics
#### i. Count of log records

In [74]:
log_data.count()

3460245

#### ii. Count of unique hosts

In [75]:
log_data.select("remotehost").distinct().count()

137978

#### iii. Date wise unique host counts

In [76]:
log_data = log_data.withColumn("date", date_format("date2", "dd-MMM-yyyy"))

In [77]:
log_data.select("date", "date2").show(10,0)

+-----------+-------------------+
|date       |date2              |
+-----------+-------------------+
|01-Jul-1995|1995-07-01 00:02:57|
|01-Jul-1995|1995-07-01 00:03:33|
|01-Jul-1995|1995-07-01 00:07:30|
|01-Jul-1995|1995-07-01 00:07:36|
|01-Jul-1995|1995-07-01 00:09:18|
|01-Jul-1995|1995-07-01 00:10:04|
|01-Jul-1995|1995-07-01 00:11:32|
|01-Jul-1995|1995-07-01 00:11:40|
|01-Jul-1995|1995-07-01 00:17:52|
|01-Jul-1995|1995-07-01 00:22:32|
+-----------+-------------------+
only showing top 10 rows



In [78]:
log_data.printSchema()

root
 |-- remotehost: string (nullable = true)
 |-- date: string (nullable = true)
 |-- request: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- bytes: integer (nullable = true)
 |-- date2: timestamp (nullable = true)



In [79]:
daily_hosts = log_data.groupBy("date").agg(countDistinct("remotehost").alias("host_count")) \
.withColumn("date_ddMMMyyyy", to_date(col("date"), "dd-MMM-yyyy")) \
.orderBy("date_ddMMMyyyy")

In [80]:
daily_hosts.select("date", "host_count").show(60)

+-----------+----------+
|       date|host_count|
+-----------+----------+
|01-Jul-1995|      5192|
|02-Jul-1995|      4859|
|03-Jul-1995|      7336|
|04-Jul-1995|      5524|
|05-Jul-1995|      7383|
|06-Jul-1995|      7820|
|07-Jul-1995|      6474|
|08-Jul-1995|      2898|
|09-Jul-1995|      2554|
|10-Jul-1995|      4464|
|11-Jul-1995|      4927|
|12-Jul-1995|      5345|
|13-Jul-1995|      6951|
|14-Jul-1995|      5297|
|15-Jul-1995|      3116|
|16-Jul-1995|      3013|
|17-Jul-1995|      4943|
|18-Jul-1995|      4523|
|19-Jul-1995|      4919|
|20-Jul-1995|      4729|
|21-Jul-1995|      4339|
|22-Jul-1995|      2575|
|23-Jul-1995|      2635|
|24-Jul-1995|      4298|
|25-Jul-1995|      4376|
|26-Jul-1995|      4296|
|27-Jul-1995|      4368|
|28-Jul-1995|      2175|
|01-Aug-1995|      2582|
|03-Aug-1995|      3222|
|04-Aug-1995|      4191|
|05-Aug-1995|      2502|
|06-Aug-1995|      2538|
|07-Aug-1995|      4108|
|08-Aug-1995|      4406|
|09-Aug-1995|      4317|
|10-Aug-1995|      4523|


#### iv. Average Requests per Host per Day

In [81]:
log_data = log_data.withColumn("dayofmonth", dayofmonth(col("date2")))

In [82]:
log_data.select("date2", "dayofmonth").show(10,0)

+-------------------+----------+
|date2              |dayofmonth|
+-------------------+----------+
|1995-07-01 00:02:57|1         |
|1995-07-01 00:03:33|1         |
|1995-07-01 00:07:30|1         |
|1995-07-01 00:07:36|1         |
|1995-07-01 00:09:18|1         |
|1995-07-01 00:10:04|1         |
|1995-07-01 00:11:32|1         |
|1995-07-01 00:11:40|1         |
|1995-07-01 00:17:52|1         |
|1995-07-01 00:22:32|1         |
+-------------------+----------+
only showing top 10 rows



In [83]:
requests_per_host_per_day = log_data.groupBy("dayofmonth").agg(
    count("request").alias("total_requests"),
    countDistinct("remotehost").alias("total_hosts")
).withColumn("avg_requests_per_host", col("total_requests") / col("total_hosts"))

In [84]:
requests_per_host_per_day = requests_per_host_per_day.orderBy("dayofmonth")

In [85]:
requests_per_host_per_day.select("dayofmonth", "avg_requests_per_host").show(40, 0)

+----------+---------------------+
|dayofmonth|avg_requests_per_host|
+----------+---------------------+
|1         |12.960341431385423   |
|2         |12.399053303148795   |
|3         |12.777723545490042   |
|4         |13.803929899097186   |
|5         |13.111468270427208   |
|6         |13.149112426035503   |
|7         |14.375969763278297   |
|8         |13.909793452297317   |
|9         |14.275995822765926   |
|10        |15.705248359887536   |
|11        |16.106119199272065   |
|12        |16.03341112885395    |
|13        |17.822921019640617   |
|14        |15.464654061022776   |
|15        |14.63315103071098    |
|16        |14.624300111982082   |
|17        |15.142017186793305   |
|18        |14.571238509917755   |
|19        |14.465010351966873   |
|20        |14.053985302430752   |
|21        |14.93733681462141    |
|22        |13.615373352855052   |
|23        |14.291256429096252   |
|24        |14.682344064386317   |
|25        |14.388222595346607   |
|26        |13.40990

#### v. Number of 404 Response Codes

In [86]:
not_found_responses = log_data.filter(col("status") == 404)

In [87]:
num_not_found = not_found_responses.count()

In [88]:
print(f"Number of 404 responses: {num_not_found}")

Number of 404 responses: 20804


#### vi. Top 15 Endpoints with 404 Responses

 - We did some analysis on request column to extract the endpoint.
 - After analyzing, we came to the conclusion that endpoint3 is the best indicator of the actual endpoint value.
 - We will proceed with endpoint3 and use it to solve the given problem.

In [89]:
not_found_endpoints = not_found_responses.withColumn(
    "endpoint",
    regexp_extract(col("request"), "^[A-Z]+\\s+([^\\s]+)\\s+HTTP", 1)
).withColumn("endpoint2", split(col("request"), " ").getItem(1))

In [90]:
not_found_endpoints.select("request", "endpoint", "endpoint2").show(20,0)

+--------------------------------------------------------------+-------------------------------------------------+-------------------------------------------------+
|request                                                       |endpoint                                         |endpoint2                                        |
+--------------------------------------------------------------+-------------------------------------------------+-------------------------------------------------+
|GET /cgi-bin/imagemap/~tfish/oregon.map?445,139 HTTP/1.0      |/cgi-bin/imagemap/~tfish/oregon.map?445,139      |/cgi-bin/imagemap/~tfish/oregon.map?445,139      |
|GET /shutle/missions/sts-71/mission-sts-71.html HTTP/1.0      |/shutle/missions/sts-71/mission-sts-71.html      |/shutle/missions/sts-71/mission-sts-71.html      |
|GET /pub/winvn/release.txt HTTP/1.0                           |/pub/winvn/release.txt                           |/pub/winvn/release.txt                           |
|GET /hist

In [91]:
not_found_endpoints = not_found_endpoints.withColumn(
       "endpoint3",
       regexp_replace(col("request"), "^(GET|POST)\\s+|\\s+HTTP/1.0$", "")
   )

In [92]:
not_found_endpoints.filter("endpoint != endpoint2").select("endpoint", "request", "endpoint2").distinct().show(100,0)

+-------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------+
|endpoint                       |request                                                                                                                                                                                                                                                                                                                                      |endpoint2                                                                             |
+-------------------------------+---------------------------------------------------------

In [93]:
not_found_endpoints.filter("endpoint2 != endpoint3").select("request", "endpoint", "endpoint2", "endpoint3").show(20,0)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+---------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------+
|request                                                                                                                                                          |endpoint|endpoint2                                    |endpoint3                                                                                                                                           |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+---------------------------------------------+----------------------------------------------

In [94]:
endpoint_counts = not_found_endpoints.groupBy("endpoint3").agg(
    count("*").alias("error_count")
).orderBy(col("error_count").desc())

In [95]:
endpoint_counts.show(15, truncate=False)

+-----------------------------------------------------------------+-----------+
|endpoint3                                                        |error_count|
+-----------------------------------------------------------------+-----------+
|/pub/winvn/readme.txt                                            |2003       |
|/pub/winvn/release.txt                                           |1732       |
|/shuttle/missions/STS-69/mission-STS-69.html                     |683        |
|/shuttle/missions/sts-68/ksc-upclose.gif                         |428        |
|/history/apollo/sa-1/sa-1-patch-small.gif                        |382        |
|/history/apollo/a-001/a-001-patch-small.gif                      |382        |
|/://spacelink.msfc.nasa.gov                                      |381        |
|/images/crawlerway-logo.gif                                      |374        |
|/elv/DELTA/uncons.htm                                            |372        |
|/history/apollo/pad-abort-test-1/pad-ab

#### vii. Top 15 Hosts with 404 Responses

In [96]:
host_error_counts = not_found_responses.groupBy("remotehost").agg(
    count("*").alias("error_count")
).orderBy(col("error_count").desc())

In [97]:
host_error_counts.show(15, 0)

+---------------------------+-----------+
|remotehost                 |error_count|
+---------------------------+-----------+
|hoohoo.ncsa.uiuc.edu       |251        |
|piweba3y.prodigy.com       |156        |
|jbiagioni.npt.nuwc.navy.mil|132        |
|piweba1y.prodigy.com       |114        |
|www-d4.proxy.aol.com       |90         |
|piweba4y.prodigy.com       |84         |
|scooter.pa-x.dec.com       |69         |
|www-d1.proxy.aol.com       |64         |
|dialip-217.den.mmc.com     |62         |
|www-b4.proxy.aol.com       |62         |
|www-b3.proxy.aol.com       |61         |
|www-a2.proxy.aol.com       |60         |
|piweba2y.prodigy.com       |59         |
|titan02f                   |59         |
|www-d2.proxy.aol.com       |59         |
+---------------------------+-----------+
only showing top 15 rows

